## **Rest Server**

In [2]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path

REDIS_CONNECTION_ARGS = {
    'host': '****',
    'port': '14517',
    'user': 'default',
    'password': '****',
}

`Redis Client` connection:

In [3]:
redis_client = redis.Redis(
    host=REDIS_CONNECTION_ARGS['host'], 
    port=REDIS_CONNECTION_ARGS['port'], 
    username=REDIS_CONNECTION_ARGS['user'], 
    password=REDIS_CONNECTION_ARGS['password'])

is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


`Endpoint / Devices` 
- `GET` - Retrieve the list of MAC addresses of the monitored devices

In [4]:
class Devices(object):
    exposed = True

    def GET(self, *path, **query):

        keys = redis_client.keys('*:*')
        devices = []
        
        for key in keys:
            device = key.decode().split(":")[0]
            if device not in devices:
                devices.append(device)

        response_dict = {
            'mac_addresses': devices
        }

        response = json.dumps(response_dict)
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        return response

`Endpoint / Device / {mac_address}`
- `GET` - Retrieve the battery status information of the device with the specified MAC address in the specified time range,
- `DELETE` - Delete the timeseries associated to the specified MAC address.

In [7]:
class Device(object):
    exposed = True

    # device/{mac_adress} - retrieve battery info in specified time range
    def GET(self, *path, **query):

        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad request: missing MAC address')

        from_timestamp = query.get('from')
        to_timestamp = query.get('to')

        if(from_timestamp is None):
            raise cherrypy.HTTPError(400, 'Bad request: missing start time')

        if(to_timestamp is None):
            raise cherrypy.HTTPError(400, 'Bad request: missing end time')

        
        mac_address = path[0]
        battery_levels = []
        power_plugged = []
        timestamps = []

        # BatteryStatus data retrieving
        try:
            # POWER
            ts_power = redis_client.ts().range(
                key=str(mac_address+":power"),
                from_time=from_timestamp,
                to_time=to_timestamp
            )
            power_plugged = [int(power[1]) for power in ts_power]
            
            # BATTERY
            ts_battery = redis_client.ts().range(
                key=str(mac_address+":battery"),
                from_time=from_timestamp,
                to_time=to_timestamp
            )
            battery_levels = [battery[1] for battery in ts_battery]
            
            timestamps = [tmst[0] for tmst in ts_battery]

        except redis.exceptions.ResponseError:
            cherrypy.response.status = "400 - Not found: invalid MAC address"
            raise cherrypy.HTTPError(404, 'Not found: invalid MAC address')


        # Response
        response_dict = {
            'mac_addresses': mac_address,
            'timestamps' : timestamps,
            'battery_levels' : battery_levels,
            'power_pluggeed' : power_plugged,
        }

        response = json.dumps(response_dict,indent=0)
        cherrypy.response.status = "200 - OK: Everything worked as expected."
        return response

    # device/{mac_adress} - delete ts associated with specified MAC address
    def DELETE(self, *path, **query):

        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad request: missing MAC address')

        mac_address = path[0]
        #to_timestamp = redis_client.ts().get(str(mac_address+":power"))[0]
        # Timeseries DELETE
        if(redis_client.exists(str(mac_address+":battery"))
            or redis_client.exists(str(mac_address+":power"))):
            redis_client.delete(str(mac_address+":battery"))
            redis_client.delete(str(mac_address+":power"))
        else:
            raise cherrypy.HTTPError(404, 'Not found: invalid MAC address')

        cherrypy.response.status = "200 - OK: Everything worked as expected."
        return

**To enable the API**: in the `Project` tab, go to `Environment`, then click on the wheel icon and enable `Allow incoming connection`. The API will be accessible through the indicated tunnelling link.


In [9]:
if __name__ == '__main__':
    conf = {'/': {
        'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
        'tools.sessions.on': True,
        }}
    cherrypy.tree.mount(Devices(), '/devices', conf)
    cherrypy.tree.mount(Device(), '/device', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[19/Jan/2023:15:58:34] ENGINE Bus STARTING
[19/Jan/2023:15:58:34] ENGINE Started monitor thread 'Autoreloader'.
[19/Jan/2023:15:58:34] ENGINE Serving on http://0.0.0.0:8080
[19/Jan/2023:15:58:34] ENGINE Bus STARTED
172.3.22.214 - - [19/Jan/2023:15:58:39] "GET /devices HTTP/1.1" 200 37 "" "python-requests/2.28.1"
172.3.22.214 - - [19/Jan/2023:15:59:30] "GET /device/0x88d82e5dbd7a?from=1672417445017&to=1674143964034 HTTP/1.1" 200 12996 "" "python-requests/2.28.1"
172.3.150.40 - - [19/Jan/2023:16:02:16] "GET /device/0x88d82e5dbd7a?from=1672417445017&to=1674143964034 HTTP/1.1" 200 12996 "" "python-requests/2.28.1"
172.3.133.85 - - [19/Jan/2023:16:03:07] "DELETE /device/0x88d82e5dbd7a HTTP/1.1" 200 - "" "python-requests/2.28.1"
172.3.133.85 - - [19/Jan/2023:16:03:56] "GET /devices HTTP/1.1" 200 21 "" "PostmanRuntime/7.30.0"
172.3.133.85 - - [19/Jan/2023:16:09:38] "GET /devices HTTP/1.1" 200 37 "" "PostmanRuntime/7.30.0"
172.3.22.214 - - [19/Jan/2023:16:09:48] "GET /device/0x2ea63389558f?fro

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=28d88834-5f02-41a4-b15c-b006e84e3419' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>